In [1]:
##laod packages
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
import numpy as np
from scipy.signal import butter, lfilter
import librosa
import soundfile as sf
import pandas as pd

In [2]:
#build the data load function 
def load_wav_48k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    return wav

In [3]:
#build preprocess function
def preprocess(wav, label):
    # Normalize the audio to the range [-1, 1]
    wav = wav / tf.reduce_max(tf.abs(wav))
    # Ensure the audio is of length 15000 (pad if shorter, trim if longer)
    wav = wav[:15000]  # Trim to 15000 samples
    zero_padding = tf.zeros([15000] - tf.shape(wav), dtype=tf.float32)  # Padding
    wav = tf.concat([zero_padding, wav], 0)
    # Apply STFT
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    # Expand dimensions (add channel dimension)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

In [4]:
# Function to process and predict on the segments with thresholding
def process_and_predict(FOREST_FILE, threshold=0.9):
    # Load the audio file
    audio = load_wav_48k_mono(FOREST_FILE)  
    
    # Set the desired length for each segment (same as input data, 15,000 samples per segment)
    segment_length = 15000  
    num_segments = len(audio) // segment_length  # Integer division

    # Slice the audio into segments of 15,000 samples each
    slices = [audio[i * segment_length: (i + 1) * segment_length] for i in range(num_segments)]

    # Preprocess each segment
    processed_segments = []
    for segment in slices:
        spectrogram, _ = preprocess(segment, label=None)
        processed_segments.append(spectrogram)

    # Convert to numpy array (or tensor) for prediction
    X_test = np.array(processed_segments)  

    # Make predictions
    predictions = model.predict(X_test)  

    # Apply thresholding: if probability > threshold, classify as 1 (frog), otherwise 0 (no frog)
    predicted_labels = (predictions > threshold).astype(int)
    
    # Count the number of segments predicted as 1 (i.e., frog detected)
    num_frog_segments = np.sum(predicted_labels)

    # Determine if the file has a frog based on the threshold
    has_frog = 1 if num_frog_segments > 0 else 0

    return has_frog, num_frog_segments

In [8]:
from tensorflow.keras.models import load_model
import tensorflow as tf

# Load the model with the custom loss function
model = load_model('strong_input.keras')

In [50]:
def process_multiple_files(directory, threshold=0.9):
    results = []

    # Loop through all the files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        try:
            # Try processing the file
            has_frog, num_frog_segments = process_and_predict(file_path, threshold)
        except Exception as e:
            # If an error occurs, mark result as N/A 
            print(f"Skipping or marking as N/A: {filename} (Error: {e})")
            has_frog = "N/A"
            num_frog_segments = "N/A"
        
        # Append the results to the list
        results.append({
            'File Name': filename,
            'Has Frog': has_frog,
            'Number of Frog Calls': num_frog_segments
        })

    # Convert the results to DataFrame
    df = pd.DataFrame(results)

    # Save the results to a CSV file
    output_csv = os.path.join(directory, 'ValeCandirus_site14.csv')
    df.to_csv(output_csv, index=False)

    print(f"Results saved to {output_csv}")

In [51]:
# go through all the files
directory = 'ValeCandirus_Site14'  # Set the path to your directory containing the .WAV files
process_multiple_files(directory, threshold=0.9)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6/6 ━━━━━━━━